In [1]:
from pymongo  import MongoClient
from wordcloud import WordCloud
import pandas as pd
import re
import collections
from pathlib import Path
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn import decomposition, naive_bayes, preprocessing, model_selection, metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Connexion à la BDD

In [2]:
client = MongoClient(host="localhost", port=27017)
db = client["PLDAC"]
avis = db["avis"]
details = db["details"]

In [3]:
df_avis = pd.DataFrame(list(avis.find()))
df_details = pd.DataFrame(list(details.find()))

In [4]:
df_avis

,_id,author,date_published,title_review,note,review_href,title,url,treated,comment
0,63c6726ea81f3e3016af9a56,Monsieur Guillaume,2021-01-27 11:06:44,Voyages sur les ailes des papillons,8.0,https://www.trictrac.net/avis/vraiment-bon-12514,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,True,"Lorsque le jeu est jeu, bon, réflexif, joli po..."
1,63c6726ea81f3e3016af9a57,morlockbob,2020-10-18 10:04:21,le festival de Michoacan,7.0,https://www.trictrac.net/avis/le-festival-de-m...,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,True,Comment continuer après un mega hit ? Simpleme...
2,63c6726ea81f3e3016af9a58,SwatSh,2021-02-01 08:35:08,Vivez la migration des monarques,7.0,https://www.trictrac.net/avis/vivez-la-migrati...,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,True,"Vin d'jeu: Avec Mariposas, Elizabeth Hargrave ..."
3,63c6726ea81f3e3016af9a59,Timi JeuxATheme,2020-11-19 17:04:57,Bon,8.0,None,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,True,NaN
4,63c6726ea81f3e3016af9a5a,prunelles,2021-05-24 12:43:18,Envolez-moi,9.0,https://www.trictrac.net/avis/envolez-moi,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,True,"Très joli bijou que ce jeu-là ! Le matériel, l..."
...,...,...,...,...,...,...,...,...,...,...
246519,63c7f00aca188a9a85b03343,ToMAHaWK,2007-03-23 12:28:45,Les Zombies à la ville,10.0,https://www.trictrac.net/avis/les-zombies-a-la...,Zombie Town,https://www.trictrac.net/jeu-de-societe/zombie...,True,Twilight revient à ce qu'ils savent faire de m...
246520,63c7f00aca188a9a85b03347,Zombie Killer,2007-09-29 12:28:47,Un bon jeu tactique,8.0,https://www.trictrac.net/avis/un-bon-jeu-tactique,Zombie Town,https://www.trictrac.net/jeu-de-societe/zombie...,True,J'ai eu l'occasion de tester ce jeu en deux oc...
246521,63c7f00aca188a9a85b0334b,sylla,2012-04-24 12:29:04,"Le ""Can't Stop"" des campus américains",6.0,https://www.trictrac.net/avis/le-can-t-stop-de...,Cosmic Wimpout,https://www.trictrac.net/jeu-de-societe/cosmic...,True,Cosmic Wimpout est né dans les années 70 sur l...
246522,63c7f00cca188a9a85b0334f,crashphil,2007-04-20 12:28:46,Simplifiez vous la vie,8.0,https://www.trictrac.net/avis/simplifiez-vous-...,Simple GBoH,https://www.trictrac.net/jeu-de-societe/simple...,False,NaN


### Prétraitements 

In [5]:
def delete_digit(doc):
    return re.sub('[0-9]+', '', doc)

In [6]:
import string

def delete_ponctuation(doc):
    punc = string.punctuation 
    punc += '\n\r\t'
    return doc.translate(str.maketrans(punc, ' ' * len(punc)))

In [7]:
comment = df_avis['comment'].astype(str)

In [8]:
comment = comment.str.lower()

In [9]:
comment = comment.map(delete_digit)

In [10]:
comment = comment.map(delete_ponctuation)

In [11]:
comment

0         lorsque le jeu est jeu  bon  réflexif  joli po...
1         comment continuer après un mega hit   simpleme...
2         vin d jeu  avec mariposas  elizabeth hargrave ...
3                                                       nan
4         très joli bijou que ce jeu là   le matériel  l...
                                ...                        
246519    twilight revient à ce qu ils savent faire de m...
246520    j ai eu l occasion de tester ce jeu en deux oc...
246521    cosmic wimpout est né dans les années  sur les...
246522                                                  nan
246523                                                  nan
Name: comment, Length: 246524, dtype: object

### Classifiers

In [12]:
y = df_avis['note'].round() # les notes sont arrondies pour créer 11 labels 0 -> 10 

In [13]:
np.random.seed(0)

### régression logistique : la pondération des classes est équilibrée

In [33]:
def lvectorizer(vectorizer):
    
    
    X = vectorizer.fit_transform(comment)

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=0.2, random_state=0)

    clf = linear_model.LogisticRegression(class_weight='balanced',random_state=0, solver='lbfgs',max_iter=1000,n_jobs=-1)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    report = metrics.classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    
    print(f"Logistic Regression accuracy: {accuracy_score(y_test, y_pred)}")
   
    return report_df

### CountVectorizer

In [34]:
def clvectorizer(**count_vectorizer_args):
    vectorizer = CountVectorizer(**count_vectorizer_args)
    return lvectorizer(vectorizer)

### TfidfVectorizer

In [35]:
def tlvectorizer(**tfidf_vectorizer_args):
    vectorizer = TfidfVectorizer(**tfidf_vectorizer_args)
    return lvectorizer(vectorizer)

### Sans arguments 

In [27]:
clvectorizer()

Logistic Regression accuracy: 0.39868167528648213


,precision,recall,f1-score,support
0.0,0.032916,0.625000,0.062539,160.000000
1.0,0.335260,0.444444,0.382208,261.000000
2.0,0.361905,0.449861,0.401118,1436.000000
3.0,0.334532,0.454768,0.385492,409.000000
4.0,0.382766,0.437787,0.408431,3054.000000
5.0,0.129911,0.738122,0.220936,1810.000000
6.0,0.451295,0.399494,0.423817,7109.000000
7.0,0.447843,0.298562,0.358275,3825.000000
8.0,0.554594,0.327110,0.411506,13711.000000
9.0,0.452712,0.317355,0.373138,5445.000000


In [28]:
tlvectorizer()

Logistic Regression accuracy: 0.3632897272081939


,precision,recall,f1-score,support
0.0,0.237197,0.550000,0.331450,160.00000
1.0,0.271689,0.455939,0.340486,261.00000
2.0,0.297154,0.509053,0.375257,1436.00000
3.0,0.251989,0.464548,0.326741,409.00000
4.0,0.313746,0.402096,0.352468,3054.00000
5.0,0.114636,0.640884,0.194484,1810.00000
6.0,0.401796,0.358841,0.379105,7109.00000
7.0,0.340842,0.181961,0.237259,3825.00000
8.0,0.523422,0.277077,0.362344,13711.00000
9.0,0.279054,0.307805,0.292726,5445.00000


### stopswords

In [14]:
stops_words_french = stopwords.words('french')

In [30]:
clvectorizer(stop_words=stops_words_french)

Logistic Regression accuracy: 0.4105060338708042


,precision,recall,f1-score,support
0.0,0.032184,0.612500,0.061154,160.000000
1.0,0.319444,0.440613,0.370370,261.000000
2.0,0.366111,0.458914,0.407293,1436.000000
3.0,0.345588,0.459658,0.394544,409.000000
4.0,0.390835,0.438441,0.413272,3054.000000
5.0,0.131312,0.744199,0.223235,1810.000000
6.0,0.458703,0.407793,0.431752,7109.000000
7.0,0.472190,0.317386,0.379612,3825.000000
8.0,0.576000,0.341332,0.428650,13711.000000
9.0,0.482557,0.332782,0.393913,5445.000000


In [31]:
tlvectorizer(stop_words=stops_words_french)

Logistic Regression accuracy: 0.36422269546699115


,precision,recall,f1-score,support
0.0,0.230352,0.531250,0.321361,160.000000
1.0,0.277108,0.440613,0.340237,261.000000
2.0,0.293014,0.511142,0.372494,1436.000000
3.0,0.259259,0.479218,0.336481,409.000000
4.0,0.321594,0.409627,0.360311,3054.000000
5.0,0.117630,0.659116,0.199632,1810.000000
6.0,0.404533,0.361514,0.381815,7109.000000
7.0,0.347623,0.195033,0.249874,3825.000000
8.0,0.522857,0.274451,0.359958,13711.000000
9.0,0.281116,0.312764,0.296097,5445.000000


### stopswords et les 1000 termes les plus fréquents

In [23]:
clvectorizer(stop_words=stops_words_french,max_features=1000)

Logistic Regression accuracy: 0.25960855896967855


,precision,recall,f1-score,support
0.0,0.042633,0.425000,0.077493,160.000000
1.0,0.049483,0.348659,0.086667,261.000000
2.0,0.082429,0.327994,0.131748,1436.000000
3.0,0.055381,0.259169,0.091261,409.000000
4.0,0.223684,0.222659,0.223170,3054.000000
5.0,0.095941,0.541989,0.163025,1810.000000
6.0,0.340040,0.261500,0.295642,7109.000000
7.0,0.207146,0.112157,0.145522,3825.000000
8.0,0.471338,0.199694,0.280533,13711.000000
9.0,0.237466,0.128742,0.166964,5445.000000


In [32]:
tlvectorizer(stop_words=stops_words_french,max_features=1000)

Logistic Regression accuracy: 0.26559172497718286


,precision,recall,f1-score,support
0.0,0.042424,0.437500,0.077348,160.000000
1.0,0.044741,0.295019,0.077699,261.000000
2.0,0.091010,0.341922,0.143756,1436.000000
3.0,0.052073,0.251834,0.086301,409.000000
4.0,0.235834,0.243942,0.239820,3054.000000
5.0,0.095873,0.536464,0.162674,1810.000000
6.0,0.349128,0.267408,0.302852,7109.000000
7.0,0.199185,0.102222,0.135107,3825.000000
8.0,0.474380,0.214718,0.295627,13711.000000
9.0,0.232300,0.130762,0.167333,5445.000000


### stopswords et tous les mots qui apparaissent dans plus de 75% des documents sont ignorés

In [33]:
clvectorizer(stop_words=stops_words_french,max_df=.75)

Logistic Regression accuracy: 0.4105060338708042


,precision,recall,f1-score,support
0.0,0.032184,0.612500,0.061154,160.000000
1.0,0.319444,0.440613,0.370370,261.000000
2.0,0.366111,0.458914,0.407293,1436.000000
3.0,0.345588,0.459658,0.394544,409.000000
4.0,0.390835,0.438441,0.413272,3054.000000
5.0,0.131312,0.744199,0.223235,1810.000000
6.0,0.458703,0.407793,0.431752,7109.000000
7.0,0.472190,0.317386,0.379612,3825.000000
8.0,0.576000,0.341332,0.428650,13711.000000
9.0,0.482557,0.332782,0.393913,5445.000000


In [34]:
tlvectorizer(stop_words=stops_words_french,max_df=.75)

Logistic Regression accuracy: 0.36422269546699115


,precision,recall,f1-score,support
0.0,0.230352,0.531250,0.321361,160.000000
1.0,0.277108,0.440613,0.340237,261.000000
2.0,0.293014,0.511142,0.372494,1436.000000
3.0,0.259259,0.479218,0.336481,409.000000
4.0,0.321594,0.409627,0.360311,3054.000000
5.0,0.117630,0.659116,0.199632,1810.000000
6.0,0.404533,0.361514,0.381815,7109.000000
7.0,0.347623,0.195033,0.249874,3825.000000
8.0,0.522857,0.274451,0.359958,13711.000000
9.0,0.281116,0.312764,0.296097,5445.000000


### stopswords et tous les mots qui apparaissent dans plus de 50% des documents sont ignorés

In [20]:
clvectorizer(stop_words=stops_words_french,max_df=.5)

Logistic Regression accuracy: 0.41081026265084675


,precision,recall,f1-score,support
0.0,0.032194,0.612500,0.061174,160.00000
1.0,0.320334,0.440613,0.370968,261.00000
2.0,0.370895,0.456128,0.409119,1436.00000
3.0,0.355805,0.464548,0.402969,409.00000
4.0,0.389148,0.436804,0.411601,3054.00000
5.0,0.131471,0.746409,0.223564,1810.00000
6.0,0.461124,0.409622,0.433850,7109.00000
7.0,0.470497,0.316863,0.378691,3825.00000
8.0,0.575694,0.341696,0.428853,13711.00000
9.0,0.483155,0.331864,0.393468,5445.00000


In [21]:
tlvectorizer(stop_words=stops_words_french,max_df=.5)

Logistic Regression accuracy: 0.3639184666869486


,precision,recall,f1-score,support
0.0,0.237057,0.543750,0.330171,160.000000
1.0,0.283951,0.440613,0.345345,261.000000
2.0,0.294118,0.508357,0.372639,1436.000000
3.0,0.255541,0.479218,0.333333,409.000000
4.0,0.321969,0.411264,0.361179,3054.000000
5.0,0.117380,0.657459,0.199197,1810.000000
6.0,0.404082,0.362076,0.381927,7109.000000
7.0,0.350375,0.195294,0.250797,3825.000000
8.0,0.520700,0.273357,0.358506,13711.000000
9.0,0.278970,0.310376,0.293836,5445.000000


### sans réequilibrage 

In [15]:
def wvectorizer(vectorizer):
    
    
    X = vectorizer.fit_transform(comment)

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=0.2, random_state=0)

    clf = linear_model.LogisticRegression(random_state=0, solver='lbfgs',max_iter=1000,n_jobs=-1)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    report = metrics.classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    
    print(f"Logistic Regression accuracy: {accuracy_score(y_test, y_pred)}")
   
    return report_df

In [16]:
def cwvectorizer(**count_vectorizer_args):
    vectorizer = CountVectorizer(**count_vectorizer_args)
    return wvectorizer(vectorizer)

In [17]:
def twvectorizer(**tfidf_vectorizer_args):
    vectorizer = TfidfVectorizer(**tfidf_vectorizer_args)
    return wvectorizer(vectorizer)

In [18]:
cwvectorizer()

Logistic Regression accuracy: 0.46528749619714027


,precision,recall,f1-score,support
0.0,0.715909,0.393750,0.508065,160.000000
1.0,0.735043,0.329502,0.455026,261.000000
2.0,0.503937,0.356546,0.417618,1436.000000
3.0,0.739796,0.354523,0.479339,409.000000
4.0,0.440758,0.365422,0.399570,3054.000000
5.0,0.650709,0.202762,0.309183,1810.000000
6.0,0.467254,0.417499,0.440978,7109.000000
7.0,0.606546,0.227712,0.331116,3825.000000
8.0,0.374746,0.658231,0.477589,13711.000000
9.0,0.592241,0.252342,0.353896,5445.000000


In [19]:
twvectorizer()

Logistic Regression accuracy: 0.42297941385255045


C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

,precision,recall,f1-score,support
0.0,0.000000,0.000000,0.000000,160.000000
1.0,1.000000,0.007663,0.015209,261.000000
2.0,0.525680,0.242340,0.331745,1436.000000
3.0,0.875000,0.017115,0.033573,409.000000
4.0,0.408597,0.295678,0.343085,3054.000000
5.0,0.700000,0.042541,0.080208,1810.000000
6.0,0.432640,0.395274,0.413114,7109.000000
7.0,0.530337,0.061699,0.110539,3825.000000
8.0,0.355773,0.694187,0.470443,13711.000000
9.0,0.488157,0.087052,0.147756,5445.000000


In [22]:
cwvectorizer(stop_words=stops_words_french)

Logistic Regression accuracy: 0.48104654700334654


,precision,recall,f1-score,support
0.0,0.714286,0.437500,0.542636,160.000000
1.0,0.770492,0.360153,0.490862,261.000000
2.0,0.516066,0.369081,0.430369,1436.000000
3.0,0.767677,0.371638,0.500824,409.000000
4.0,0.461255,0.368369,0.409612,3054.000000
5.0,0.682724,0.227072,0.340796,1810.000000
6.0,0.480981,0.430440,0.454309,7109.000000
7.0,0.652870,0.255686,0.367462,3825.000000
8.0,0.384346,0.673328,0.489359,13711.000000
9.0,0.639785,0.284114,0.393488,5445.000000


In [23]:
twvectorizer(stop_words=stops_words_french)

Logistic Regression accuracy: 0.42304025960855896


C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

,precision,recall,f1-score,support
0.0,0.000000,0.000000,0.000000,160.00000
1.0,1.000000,0.007663,0.015209,261.00000
2.0,0.532609,0.238858,0.329808,1436.00000
3.0,1.000000,0.022005,0.043062,409.00000
4.0,0.410933,0.285527,0.336940,3054.00000
5.0,0.684615,0.049171,0.091753,1810.00000
6.0,0.431177,0.395696,0.412675,7109.00000
7.0,0.544534,0.070327,0.124566,3825.00000
8.0,0.355706,0.691489,0.469763,13711.00000
9.0,0.494717,0.094582,0.158804,5445.00000


In [20]:
cwvectorizer(stop_words=stops_words_french,max_df=.5)

Logistic Regression accuracy: 0.48078288206064296


,precision,recall,f1-score,support
0.0,0.727273,0.450000,0.555985,160.000000
1.0,0.760000,0.363985,0.492228,261.000000
2.0,0.519099,0.369081,0.431420,1436.000000
3.0,0.757576,0.366748,0.494234,409.000000
4.0,0.465192,0.371971,0.413392,3054.000000
5.0,0.688027,0.225414,0.339576,1810.000000
6.0,0.480700,0.429174,0.453478,7109.000000
7.0,0.643233,0.255948,0.366187,3825.000000
8.0,0.383691,0.672963,0.488731,13711.000000
9.0,0.643750,0.283747,0.393881,5445.000000


In [21]:
twvectorizer(stop_words=stops_words_french,max_df=.5)

Logistic Regression accuracy: 0.4225129297231518


C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

,precision,recall,f1-score,support
0.0,0.000000,0.000000,0.000000,160.000000
1.0,1.000000,0.007663,0.015209,261.000000
2.0,0.525502,0.236769,0.326452,1436.000000
3.0,1.000000,0.024450,0.047733,409.000000
4.0,0.412931,0.286509,0.338295,3054.000000
5.0,0.692308,0.049724,0.092784,1810.000000
6.0,0.429490,0.394570,0.411290,7109.000000
7.0,0.555556,0.070588,0.125261,3825.000000
8.0,0.355141,0.690468,0.469035,13711.000000
9.0,0.491525,0.095868,0.160443,5445.000000


### XGBoost

In [20]:
from xgboost import XGBClassifier
cv = CountVectorizer(stop_words=stops_words_french)

X = cv.fit_transform(comment)

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=0)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

report = metrics.classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0.0,0.871795,0.212500,0.341709,160.000000
1.0,1.000000,0.137931,0.242424,261.000000
2.0,0.582222,0.182451,0.277837,1436.000000
3.0,0.943396,0.122249,0.216450,409.000000
4.0,0.433708,0.189587,0.263841,3054.000000
5.0,0.861842,0.072376,0.133537,1810.000000
6.0,0.437885,0.279646,0.341317,7109.000000
7.0,0.820069,0.061961,0.115216,3825.000000
8.0,0.334700,0.750346,0.462913,13711.000000
9.0,0.712919,0.082094,0.147233,5445.000000


In [21]:
tv = TfidfVectorizer(stop_words=stops_words_french)

X = cv.fit_transform(comment)

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=0)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

report = metrics.classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0.0,0.871795,0.212500,0.341709,160.000000
1.0,1.000000,0.137931,0.242424,261.000000
2.0,0.582222,0.182451,0.277837,1436.000000
3.0,0.943396,0.122249,0.216450,409.000000
4.0,0.433708,0.189587,0.263841,3054.000000
5.0,0.861842,0.072376,0.133537,1810.000000
6.0,0.437885,0.279646,0.341317,7109.000000
7.0,0.820069,0.061961,0.115216,3825.000000
8.0,0.334700,0.750346,0.462913,13711.000000
9.0,0.712919,0.082094,0.147233,5445.000000


### Autres modeles : bigrammes, trigrammes

In [ ]:
clvectorizer(ngram_range=(1, 2), stop_words=stops_words_french)

In [ ]:
tlvectorizer(ngram_range=(1, 2), stop_words=stops_words_french)

In [ ]:
clvectorizer(ngram_range=(1, 3), stop_words=stops_words_french)

In [ ]:
tlvectorizer(ngram_range=(1, 3), stop_words=stops_words_french)

### classification binaire 

In [14]:
# decoupage (0 à 7 -> -1) (8 à 10 -> 1)
classes = [1 if note>7 else -1 for note in y]

In [15]:
classes

[1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -

In [16]:
def bvectorizer(vectorizer):
    
    
    X = vectorizer.fit_transform(comment)

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, classes, test_size=0.2, random_state=0)

    clf = linear_model.LogisticRegression(random_state=0, solver='lbfgs',max_iter=1000,n_jobs=-1)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    report = metrics.classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    
    print(f"Logistic Regression accuracy: {accuracy_score(y_test, y_pred)}")
   
    return report_df

In [17]:
def cbvectorizer(**count_vectorizer_args):
    vectorizer = CountVectorizer(**count_vectorizer_args)
    return bvectorizer(vectorizer)

In [18]:
def tbvectorizer(**tfidf_vectorizer_args):
    vectorizer = TfidfVectorizer(**tfidf_vectorizer_args)
    return bvectorizer(vectorizer)

In [21]:
cbvectorizer()

Logistic Regression accuracy: 0.7764121285873644


,precision,recall,f1-score,support
-1,0.778481,0.544730,0.640959,18064.000000
1,0.775699,0.910374,0.837658,31241.000000
accuracy,0.776412,0.776412,0.776412,0.776412
macro avg,0.777090,0.727552,0.739308,49305.000000
weighted avg,0.776718,0.776412,0.765593,49305.000000


In [22]:
tbvectorizer()

Logistic Regression accuracy: 0.766818781056688


,precision,recall,f1-score,support
-1,0.781725,0.504373,0.613143,18064.000000
1,0.762205,0.918569,0.833113,31241.000000
accuracy,0.766819,0.766819,0.766819,0.766819
macro avg,0.771965,0.711471,0.723128,49305.000000
weighted avg,0.769356,0.766819,0.752522,49305.000000


In [25]:
cbvectorizer(stop_words=stops_words_french)

Logistic Regression accuracy: 0.7741405536963797


,precision,recall,f1-score,support
-1,0.776104,0.539028,0.636197,18064.000000
1,0.773470,0.910086,0.836235,31241.000000
accuracy,0.774141,0.774141,0.774141,0.774141
macro avg,0.774787,0.724557,0.736216,49305.000000
weighted avg,0.774435,0.774141,0.762947,49305.000000


In [26]:
tbvectorizer(stop_words=stops_words_french)

Logistic Regression accuracy: 0.765338200993814


,precision,recall,f1-score,support
-1,0.779721,0.501052,0.610070,18064.000000
1,0.760909,0.918152,0.832168,31241.000000
accuracy,0.765338,0.765338,0.765338,0.765338
macro avg,0.770315,0.709602,0.721119,49305.000000
weighted avg,0.767801,0.765338,0.750797,49305.000000


In [27]:
cbvectorizer(stop_words=stops_words_french,max_df=.75)

Logistic Regression accuracy: 0.7741405536963797


,precision,recall,f1-score,support
-1,0.776104,0.539028,0.636197,18064.000000
1,0.773470,0.910086,0.836235,31241.000000
accuracy,0.774141,0.774141,0.774141,0.774141
macro avg,0.774787,0.724557,0.736216,49305.000000
weighted avg,0.774435,0.774141,0.762947,49305.000000


In [28]:
tbvectorizer(stop_words=stops_words_french,max_df=.75)

Logistic Regression accuracy: 0.765338200993814


,precision,recall,f1-score,support
-1,0.779721,0.501052,0.610070,18064.000000
1,0.760909,0.918152,0.832168,31241.000000
accuracy,0.765338,0.765338,0.765338,0.765338
macro avg,0.770315,0.709602,0.721119,49305.000000
weighted avg,0.767801,0.765338,0.750797,49305.000000


In [29]:
cbvectorizer(ngram_range=(1, 2), stop_words=stops_words_french)

Logistic Regression accuracy: 0.8173207585437582


,precision,recall,f1-score,support
-1,0.847358,0.611548,0.710395,18064.000000
1,0.806524,0.936302,0.866581,31241.000000
accuracy,0.817321,0.817321,0.817321,0.817321
macro avg,0.826941,0.773925,0.788488,49305.000000
weighted avg,0.821484,0.817321,0.809359,49305.000000


In [30]:
tbvectorizer(ngram_range=(1, 2), stop_words=stops_words_french)

Logistic Regression accuracy: 0.7871412635635331


,precision,recall,f1-score,support
-1,0.841716,0.516054,0.639830,18064.000000
1,0.771331,0.943888,0.848930,31241.000000
accuracy,0.787141,0.787141,0.787141,0.787141
macro avg,0.806523,0.729971,0.744380,49305.000000
weighted avg,0.797118,0.787141,0.772321,49305.000000


In [31]:
cbvectorizer(ngram_range=(1, 3), stop_words=stops_words_french)

Logistic Regression accuracy: 0.8195720515160734


,precision,recall,f1-score,support
-1,0.854579,0.611603,0.712958,18064.000000
1,0.807131,0.939823,0.868437,31241.000000
accuracy,0.819572,0.819572,0.819572,0.819572
macro avg,0.830855,0.775713,0.790698,49305.000000
weighted avg,0.824515,0.819572,0.811474,49305.000000


In [32]:
tbvectorizer(ngram_range=(1, 3), stop_words=stops_words_french)

Logistic Regression accuracy: 0.7855795558259812


,precision,recall,f1-score,support
-1,0.860400,0.495073,0.628505,18064.00000
1,0.765593,0.953555,0.849299,31241.00000
accuracy,0.785580,0.785580,0.785580,0.78558
macro avg,0.812997,0.724314,0.738902,49305.00000
weighted avg,0.800328,0.785580,0.768406,49305.00000
